In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

# --- 1. Configuração da Extensão e Tema ----------------------------------
pn.extension('plotly', 'tabulator')

# --- 2. CSS Global (DESIGN "GLASSMORPHISM" v2) --------------------------
background_url = 'https://m.media-amazon.com/images/S/sonata-images-prod/US_CHANNELS_NBA_LeaguePass_092225/22ef1594-50d9-459c-bf3d-7da4d8bcc873._UR1936,1089_SX2160_FMjpg_.jpeg'

pn.config.raw_css.append(f"""
body {{
    background-image: linear-gradient(rgba(0, 0, 0, 0.6), rgba(0, 0, 0, 0.6)), url('{background_url}');
    background-size: cover;
    background-attachment: fixed;
    background-position: center;
    color: #fff;
}}

/* Tornar o fundo do template transparente */
.pn-template, .pn-main {{
    background: transparent !important;
}}

/* A classe do nosso "Painel de Vidro" (para conteúdo principal) */
.glass-pane {{
    background: rgba(25, 35, 50, 0.75);
    backdrop-filter: blur(10px) saturate(120%);
    -webkit-backdrop-filter: blur(10px) saturate(120%);
    border-radius: 16px;
    padding: 24px;
    box-shadow: 0 4px 30px rgba(0, 0, 0, 0.2);
    border: 1px solid rgba(255, 255, 255, 0.1);
    margin: 20px;
}}

/* Estilo para os cartões de filtro (CORRIGIDO) */
.pn-card {{
    background: rgba(15, 23, 42, 0.7) !important; /* Fundo vidro */
    border: 1px solid rgba(255, 255, 255, 0.1) !important;
    backdrop-filter: blur(5px);
    border-radius: 8px !important;
}}
.pn-card-header {{
    background: rgba(255, 255, 255, 0.1) !important;
    border-bottom: 1px solid rgba(255, 255, 255, 0.1) !important;
}}
.pn-card-body {{
    background: transparent !important;
}}

/* CSS customizado do dashboard de Jogo */
.jogo-row {{
    background: linear-gradient(135deg, rgba(0,30,60,0.6), rgba(0,20,40,0.4));
    padding: 18px;
    border-radius: 20px;
    box-shadow: 0 8px 30px rgba(0,0,0,0.5);
    align-items: start;
}}
.jogo-row .bk-root .bk-layout-cell {{
    padding: 8px;
}}
""")

# --- 3. Carregamento e Preparo Centralizado dos Dados --------------------
def load_data():
    """
    Carrega e prepara todos os DataFrames de uma só vez.
    Isso evita cargas redundantes em diferentes abas.
    """
    try:
        data = {}
        # Carregar todos os CSVs
        data['game'] = pd.read_csv('csv/game.csv', parse_dates=['game_date'], low_memory=False)
        data['team'] = pd.read_csv('csv/team.csv', low_memory=False)
        data['line_score'] = pd.read_csv('csv/line_score.csv', parse_dates=['game_date_est'], low_memory=False)
        data['game_info'] = pd.read_csv('csv/game_info.csv', parse_dates=['game_date'], low_memory=False)
        data['player_info'] = pd.read_csv('csv/common_player_info.csv', low_memory=False)
        data['player'] = pd.read_csv('csv/player.csv', low_memory=False)

        # Padronizar colunas minúsculas
        for key in data:
            data[key].columns = [col.lower() for col in data[key].columns]

        # --- Pré-processamento: Análise de Times ---
        team_df = data['team'].rename(columns={'id': 'team_id', 'full_name': 'team_name'})
        game_df = data['game']
        
        home_df = game_df[['game_date', 'team_id_home', 'wl_home', 'fgm_home', 'fga_home', 'pts_home', 'ast_home', 'reb_home', 'game_id']].copy()
        away_df = game_df[['game_date', 'team_id_away', 'wl_away', 'fgm_away', 'fga_away', 'pts_away', 'ast_away', 'reb_away', 'game_id']].copy()

        home_df.rename(columns={
            'team_id_home': 'team_id', 'wl_home': 'wl', 'fgm_home': 'fgm', 'fga_home': 'fga',
            'pts_home': 'pts', 'ast_home': 'ast', 'reb_home': 'reb'
        }, inplace=True)
        home_df['location'] = 'Casa'

        away_df.rename(columns={
            'team_id_away': 'team_id', 'wl_away': 'wl', 'fgm_away': 'fgm', 'fga_away': 'fga',
            'pts_away': 'pts', 'ast_away': 'ast', 'reb_away': 'reb'
        }, inplace=True)
        away_df['location'] = 'Fora'

        df_combined = pd.concat([home_df, away_df], ignore_index=True)
        df_combined = pd.merge(df_combined, team_df[['team_id', 'team_name', 'abbreviation']], on='team_id', how='left')
        df_combined['year'] = df_combined['game_date'].dt.year
        df_combined['game_date_only'] = df_combined['game_date'].dt.date
        data['team_analysis_df'] = df_combined
        
        data['team_df_processed'] = team_df

        # --- Pré-processamento: Visão Geral ---
        num_cols = ['pts_home','pts_away','ast_home','ast_away','fgm_home','fga_home','fgm_away','fga_away']
        for c in num_cols:
            if c in data['game'].columns:
                data['game'][c] = pd.to_numeric(data['game'][c], errors='coerce')
            if c in data['line_score'].columns:
                 data['line_score'][c] = pd.to_numeric(data['line_score'][c], errors='coerce')
        
        data['line_merged'] = data['line_score'].merge(data['game'][['game_id','team_name_home','team_name_away']], on='game_id', how='left')
        data['line_merged']['year'] = pd.to_datetime(data['line_merged']['game_date_est'], errors='coerce').dt.year

        # --- Pré-processamento: Análise Geográfica ---
        if 'latitude' not in data['team'].columns or 'longitude' not in data['team'].columns:
            cities = data['team']['city'].unique()
            coords = pd.DataFrame({'city': cities, 'latitude': np.random.uniform(25, 50, len(cities)), 'longitude': np.random.uniform(-125, -70, len(cities))})
            data['team'] = pd.merge(data['team'], coords, on='city', how='left')

        # --- Pré-processamento: Análise de Jogos ---
        def get_team_nickname(team_id, teams_df):
            row = teams_df[teams_df['team_id'] == team_id]
            if not row.empty:
                return row.iloc[0]['nickname']
            return ''
        
        game_with_names = data['game'].copy()
        game_with_names['nome_jogo'] = game_with_names.apply(
            lambda row: f'{get_team_nickname(row["team_id_home"], team_df)} vs. {get_team_nickname(row["team_id_away"], team_df)} ({row["game_date"].strftime("%d/%m/%Y")})', 
            axis=1
        )
        data['game_with_names'] = game_with_names

        # --- Pré-processamento: Perfil Físico ---
        def parse_height(h):
            if isinstance(h, str) and '-' in h:
                ft, inch = h.split('-')
                return round(int(ft) * 0.3048 + int(inch) * 0.0254, 2)
            return None
        data['player_info']['altura_m'] = data['player_info']['height'].apply(parse_height)
        data['player_info']['peso_kg'] = pd.to_numeric(data['player_info']['weight'], errors='coerce')

        return data
    
    except Exception as e:
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(f"ERRO AO CARREGAR OS DADOS: {e}")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        pn.pane.Alert(f"Erro ao carregar os arquivos CSV: {e}", alert_type='danger').servable()
        return None

# --- 4. Funções de Criação de Abas (Modulares) -------------------------

def create_home_tab():
    html = '''
    <div style="position:relative;width:100%;height:600px;display:flex;flex-direction:column;justify-content:center;align-items:center;">
      <div style="background:rgba(0,0,0,0.6);padding:40px 60px;border-radius:16px;color:#fff;box-shadow:0 4px 24px #0006;">
        <h1 style="margin-bottom:16px;font-size:2.5em;">Dashboard NBA</h1>
        <h2 style="margin-bottom:24px;font-size:1.5em;">Trabalho: Análise de Dados da NBA</h2>
        <h3 style="margin-bottom:12px;">Integrantes do Grupo</h3>
        <ul style="font-size:1.2em;list-style:none;padding:0;">
          <li>Igor Gabriel Rodrigues</li>
          <li>Victor Mendonça Rodrigues</li>
          <li>Davi Farias de Freitas Manzotti</li>
          <li>Victor Hugo Queiroz Couto</li>
          <li>Rodrigo Debortoli de Souza</li>
        </ul>
      </div>
    </div>
    '''
    return pn.pane.HTML(html, sizing_mode='stretch_width', height=600)

def create_overview_tab(data):
    game_df = data['game']
    line_merged = data['line_merged']

    # --- Cálculos Globais (Todos os Times) ---
    total_games = game_df['game_id'].nunique()
    total_assists = game_df['ast_home'].sum(skipna=True) + game_df['ast_away'].sum(skipna=True)
    total_points = game_df['pts_home'].sum(skipna=True) + game_df['pts_away'].sum(skipna=True)
    avg_points_per_team_game = ((game_df['pts_home'] + game_df['pts_away']) / 2.0).mean()
    fg_made = game_df['fgm_home'].sum(skipna=True) + game_df['fgm_away'].sum(skipna=True)
    fg_att = game_df['fga_home'].sum(skipna=True) + game_df['fga_away'].sum(skipna=True)
    fg_pct = (fg_made / fg_att * 100.0) if fg_att else 0

    by_year_global = line_merged.groupby('year', dropna=True).agg({'pts_home':'sum','pts_away':'sum'}).reset_index().sort_values('year')
    by_year_global['total'] = by_year_global['pts_home'] + by_year_global['pts_away']

    # --- Widgets ---
    # CORREÇÃO DO FUTUREWARNING:
    all_teams = pd.concat([game_df['team_name_home'], game_df['team_name_away']]).dropna().unique()
    teams = sorted(all_teams)
    
    team_sel = pn.widgets.MultiChoice(
        name='Filtrar por Time (vazio = todos)', 
        options=teams, 
        placeholder='Selecione um ou mais times...'
    )

    # --- Indicadores (KPIs) ---
    kpi_avg_pts = pn.indicators.Number(name='Média Pontos/Jogo', value=avg_points_per_team_game, format='{value:,.2f}', font_size='24pt')
    kpi_assists = pn.indicators.Number(name='Total Assistências', value=total_assists, format='{value:,.0f}', font_size='24pt')
    kpi_games = pn.indicators.Number(name='Total Jogos', value=total_games, format='{value:,.0f}', font_size='24pt')
    kpi_fg_pct = pn.indicators.Number(name='FG%', value=fg_pct, format='{value:,.2f}%', font_size='24pt')
    kpi_total_pts = pn.indicators.Number(name='Total Pontos', value=total_points, format='{value:,.0f}', font_size='24pt')
    
    kpi_row = pn.Row(kpi_avg_pts, kpi_assists, kpi_games, kpi_fg_pct, kpi_total_pts, sizing_mode='stretch_width')

    # --- Funções Reativas ---
    @pn.depends(team_sel.param.value, watch=True)
    def update_kpis(selected_teams):
        if not selected_teams:
            kpi_avg_pts.value = avg_points_per_team_game
            kpi_assists.value = total_assists
            kpi_games.value = total_games
            kpi_fg_pct.value = fg_pct
            kpi_total_pts.value = total_points
        else:
            sel = set(selected_teams)
            mask_home = game_df['team_name_home'].isin(sel)
            mask_away = game_df['team_name_away'].isin(sel)
            games_sel = game_df[mask_home | mask_away]

            team_games = games_sel['game_id'].nunique()
            team_pts = games_sel.loc[mask_home, 'pts_home'].sum(skipna=True) + games_sel.loc[mask_away, 'pts_away'].sum(skipna=True)
            team_assists = games_sel.loc[mask_home, 'ast_home'].sum(skipna=True) + games_sel.loc[mask_away, 'ast_away'].sum(skipna=True)
            team_fgm = games_sel.loc[mask_home, 'fgm_home'].sum(skipna=True) + games_sel.loc[mask_away, 'fgm_away'].sum(skipna=True)
            team_fga = games_sel.loc[mask_home, 'fga_home'].sum(skipna=True) + games_sel.loc[mask_away, 'fga_away'].sum(skipna=True)
            team_fg_pct = (team_fgm / team_fga * 100.0) if team_fga else 0
            team_avg_pts = (team_pts / team_games) if team_games else 0

            kpi_avg_pts.value = team_avg_pts
            kpi_assists.value = team_assists
            kpi_games.value = team_games
            kpi_fg_pct.value = team_fg_pct
            kpi_total_pts.value = team_pts

    @pn.depends(team_sel.param.value)
    def update_chart(selected_teams):
        if not selected_teams:
            df_by_year = by_year_global.copy()
        else:
            sel = set(selected_teams)
            pts_home_by_year = line_merged[line_merged['team_name_home'].isin(sel)].groupby('year')['pts_home'].sum()
            pts_away_by_year = line_merged[line_merged['team_name_away'].isin(sel)].groupby('year')['pts_away'].sum()
            years = sorted(set(pts_home_by_year.index).union(set(pts_away_by_year.index)))
            df_by_year = pd.DataFrame({'year': years}).set_index('year')
            df_by_year['pts_home'] = pts_home_by_year.reindex(years, fill_value=0)
            df_by_year['pts_away'] = pts_away_by_year.reindex(years, fill_value=0)
            df_by_year = df_by_year.reset_index()
            df_by_year['total'] = df_by_year['pts_home'] + df_by_year['pts_away']
        
        fig_local = go.Figure()
        fig_local.add_trace(go.Bar(name='Pontos em Casa', x=df_by_year['year'], y=df_by_year['pts_home'], marker_color='#1f77b4'))
        fig_local.add_trace(go.Bar(name='Pontos Fora', x=df_by_year['year'], y=df_by_year['pts_away'], marker_color='#9467bd'))
        fig_local.add_trace(go.Scatter(name='Total Pontos', x=df_by_year['year'], y=df_by_year['total'], mode='lines+markers', marker_color='#2ca02c'))
        
        fig_local.update_layout(
            barmode='stack', title_text='Total de Pontos por Ano (Casa vs Fora)', 
            xaxis_title='Ano', yaxis_title='Pontos', 
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1), 
            template='plotly_dark', height=450,
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)'
        )
        fig_local.update_yaxes(tickformat=',')
        
        return fig_local

    # --- Layout da Aba ---
    chart_pane = pn.pane.Plotly(
        update_chart,
        config={'displayModeBar': False}, 
        sizing_mode='stretch_width'
    )
    
    return pn.Column(
        pn.pane.Markdown('## Visão Geral'),
        pn.Card(
            team_sel, 
            title="Filtros", 
            collapsed=False, 
            sizing_mode='stretch_width'
            # 'background' FOI REMOVIDO (CORRIGIDO)
        ),
        pn.Spacer(height=20),
        kpi_row,
        pn.Spacer(height=10),
        chart_pane,
        css_classes=['glass-pane'],
        sizing_mode='stretch_width'
    )

def create_team_analysis_tab(data):
    df = data['team_analysis_df']
    team_df = data['team_df_processed']
    
    team_names = sorted(df['team_name'].dropna().unique())
    min_date = df['game_date_only'].min()
    max_date = df['game_date_only'].max()

    # --- Widgets ---
    team_selector = pn.widgets.MultiChoice(
        name='Selecione os Times', 
        options=team_names, 
        value=[team_names[0], team_names[1], team_names[2]],
        placeholder='Selecione um ou mais times...'
    )
    date_slider = pn.widgets.DateRangeSlider(
        name='Intervalo de Data dos Jogos',
        start=min_date, end=max_date,
        value=(min_date, max_date)
    )
    
    # --- Funções Reativas para os Gráficos ---
    @pn.depends(team_selector.param.value, date_slider.param.value)
    def create_fg_pct_chart(teams, date_range):
        if not teams: return go.Figure().update_layout(template='plotly_dark', title='% Arremessos Convertidos por Ano', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
        filtered_df = df[(df['team_name'].isin(teams)) & (df['game_date_only'] >= date_range[0]) & (df['game_date_only'] <= date_range[1])]
        
        yearly_stats = filtered_df.groupby(['team_name', 'year']).agg({'fgm': 'sum', 'fga': 'sum'}).reset_index()
        yearly_stats['fg_pct'] = (yearly_stats['fgm'] / yearly_stats['fga'])
        
        fig = go.Figure()
        for team in teams:
            team_stats = yearly_stats[yearly_stats['team_name'] == team]
            fig.add_trace(go.Bar(x=team_stats['year'], y=team_stats['fg_pct'], name=team))
        
        fig.update_layout(
            title='% Arremessos Convertidos por Ano', xaxis_title='Ano', yaxis_title='% Convertidos',
            template='plotly_dark', barmode='group', height=450,
            plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)'
        )
        return fig

    @pn.depends(team_selector.param.value, date_slider.param.value)
    def create_totals_chart(teams, date_range):
        if not teams: return go.Figure().update_layout(template='plotly_dark', title='Total de Pontos, Assistências, etc.', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
        filtered_df = df[(df['team_name'].isin(teams)) & (df['game_date_only'] >= date_range[0]) & (df['game_date_only'] <= date_range[1])]
        
        team_stats = filtered_df.groupby('team_name').agg({
            'pts': 'sum', 'ast': 'sum', 'reb': 'sum',
            'wl': lambda x: (x == 'W').sum(), 'game_id': 'nunique'
        }).reset_index()
        team_stats.rename(columns={'wl': 'wins', 'game_id': 'games'}, inplace=True)

        fig = go.Figure()
        fig.add_trace(go.Bar(name='Total de Pontos', x=team_stats['team_name'], y=team_stats['pts'], text=team_stats['pts'], textposition='auto'))
        fig.add_trace(go.Bar(name='Total de Assistências', x=team_stats['team_name'], y=team_stats['ast'], text=team_stats['ast'], textposition='auto'))
        fig.add_trace(go.Bar(name='Total de Rebotes', x=team_stats['team_name'], y=team_stats['reb'], text=team_stats['reb'], textposition='auto'))
        fig.add_trace(go.Bar(name='Total de Vitórias', x=team_stats['team_name'], y=team_stats['wins'], text=team_stats['wins'], textposition='auto'))
        fig.add_trace(go.Bar(name='Total de Jogos', x=team_stats['team_name'], y=team_stats['games'], text=team_stats['games'], textposition='auto'))

        fig.update_layout(
            barmode='stack', title='Total de Pontos, Assistências, Rebotes, Vitórias e Jogos',
            xaxis_title='Time', yaxis_title='Total', template='plotly_dark', legend_title_text='Métricas', height=450,
            plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)'
        )
        return fig

    @pn.depends(team_selector.param.value, date_slider.param.value)
    def create_home_away_chart(teams, date_range):
        if not teams: return go.Figure().update_layout(template='plotly_dark', title='Pontos Marcados em Casa vs. Fora', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
        filtered_df = df[(df['team_name'].isin(teams)) & (df['game_date_only'] >= date_range[0]) & (df['game_date_only'] <= date_range[1])]
        
        home_away_stats = filtered_df.groupby(['team_name', 'location'])['pts'].sum().unstack(fill_value=0).reset_index()
        team_abbreviations = pd.merge(home_away_stats, team_df[['team_name', 'abbreviation']], on='team_name')['abbreviation']

        fig = go.Figure()
        fig.add_trace(go.Bar(name='Pontos em Casa', y=team_abbreviations, x=home_away_stats['Casa'], orientation='h'))
        fig.add_trace(go.Bar(name='Pontos Fora', y=team_abbreviations, x=home_away_stats['Fora'], orientation='h'))

        fig.update_layout(
            barmode='group', title='Pontos Marcados em Casa vs. Fora',
            yaxis_title='Abreviação', xaxis_title='Pontos', template='plotly_dark', legend_title_text='Local', height=450,
            plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)'
        )
        return fig

    # --- Layout da Aba ---
    filters = pn.Card(
        team_selector,
        date_slider,
        title="Filtros",
        width=300
        # 'background' FOI REMOVIDO (CORRIGIDO)
    )
    
    return pn.Column(
        pn.pane.Markdown("## Análise de Times"),
        pn.Row(filters, pn.pane.Plotly(create_fg_pct_chart, sizing_mode='stretch_width')),
        pn.Row(
            pn.pane.Plotly(create_totals_chart, sizing_mode='stretch_width'),
            pn.pane.Plotly(create_home_away_chart, sizing_mode='stretch_width')
        ),
        css_classes=['glass-pane'],
        sizing_mode='stretch_width'
    )

def create_geo_tab(data):
    team = data['team']
    game = data['game']
    game_info = data['game_info']
    
    cities = sorted(team['city'].dropna().unique())
    min_date = game_info['game_date'].min().date()
    max_date = game_info['game_date'].max().date()

    # --- Widgets ---
    city_selector = pn.widgets.MultiChoice(
        name='Selecione a Cidade', 
        options=cities, 
        value=[cities[0]], 
        placeholder='Selecione uma ou mais cidades...'
    )
    date_slider = pn.widgets.DateRangeSlider(
        name='Intervalo de Data dos Jogos', 
        start=min_date, 
        end=max_date, 
        value=(min_date, max_date)
    )

    # --- Funções Reativas ---
    @pn.depends(city_selector.param.value)
    def pontos_por_cidade(selected_cities):
        if not selected_cities: 
            fig = go.Figure().update_layout(template='plotly_dark', title='Total de Pontos por Cidade (Home)', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
            return pn.pane.Plotly(fig, config={'displayModeBar': True, 'scrollZoom': True}, height=650)
            
        teams_cidade = team[team['city'].isin(selected_cities)]
        df_merged = pd.merge(teams_cidade[['id', 'city', 'latitude', 'longitude']], game, left_on='id', right_on='team_id_home', how='left')
        pontos = df_merged.groupby(['city', 'latitude', 'longitude']).agg({'pts_home': 'sum'}).reset_index()
        
        fig = px.scatter_mapbox(pontos, lat='latitude', lon='longitude', size='pts_home', color='pts_home', 
                                hover_name='city', size_max=60, zoom=3, mapbox_style='carto-darkmatter',
                                title='Total de Pontos por Cidade (Home)')
        fig.update_layout(height=650, margin=dict(l=0, r=0, t=40, b=0), plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
        return pn.pane.Plotly(fig, config={'displayModeBar': True, 'scrollZoom': True}, height=650)

    @pn.depends(date_slider.param.value)
    def pontos_por_data(date_range):
        df_merged = pd.merge(game_info, game[['game_id','team_id_home','pts_home']], on='game_id', how='left')
        df_merged = pd.merge(df_merged, team[['id', 'city', 'latitude', 'longitude']], left_on='team_id_home', right_on='id', how='left')
        df_merged = df_merged[(df_merged['game_date'] >= pd.to_datetime(date_range[0])) & (df_merged['game_date'] <= pd.to_datetime(date_range[1]))]
        
        pontos = df_merged.groupby(['city', 'latitude', 'longitude']).agg({'pts_home': 'sum'}).reset_index()
        
        fig = px.scatter_mapbox(pontos, lat='latitude', lon='longitude', size='pts_home', color='pts_home', 
                                hover_name='city', size_max=60, zoom=3, mapbox_style='carto-darkmatter',
                                title='Total de Pontos por Cidade (Filtrado por Data)')
        fig.update_layout(height=650, margin=dict(l=0, r=0, t=40, b=0), plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
        return pn.pane.Plotly(fig, config={'displayModeBar': True, 'scrollZoom': True}, height=650)

    # --- Layout da Aba ---
    return pn.Column(
        pn.pane.Markdown('## Análise Geográfica'),
        pn.Row(
            pn.Column(
                pn.Card(city_selector, title='Filtro por Cidade'), # 'background' FOI REMOVIDO
                pontos_por_cidade,
                sizing_mode='stretch_width'
            ),
            pn.Column(
                pn.Card(date_slider, title='Filtro por Data dos Jogos'), # 'background' FOI REMOVIDO
                pontos_por_data,
                sizing_mode='stretch_width'
            ),
        ),
        css_classes=['glass-pane'],
        sizing_mode='stretch_width'
    )

def create_player_analysis_tab(data):
    players = data['player_info']
    
    schools = sorted(players['school'].dropna().unique())
    positions = sorted(players['position'].dropna().unique())

    # --- Widgets ---
    school_selector = pn.widgets.MultiChoice(
        name='Filtrar por Escola', 
        options=schools, 
        value=schools[:5],
        placeholder='Selecione uma ou mais escolas...'
    )
    position_selector = pn.widgets.MultiChoice(
        name='Filtrar por Posição', 
        options=positions, 
        value=positions,
        placeholder='Selecione uma ou mais posições...'
    )

    # --- Funções Reativas ---
    def filtrar_jogadores(selected_schools, selected_positions):
        return players[(players['school'].isin(selected_schools)) & (players['position'].isin(selected_positions))]

    @pn.depends(school_selector.param.value, position_selector.param.value)
    def grafico_school(schools, positions):
        if not schools or not positions: 
            return go.Figure().update_layout(template='plotly_dark', title='Contagem de Jogadores por Escola', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
        
        df = filtrar_jogadores(schools, positions)
        contagem_school = df.groupby('school').agg({'person_id': 'count'}).reset_index()
        contagem_school = contagem_school.sort_values('person_id', ascending=False).head(20)
        
        fig = px.bar(contagem_school, x='person_id', y='school', orientation='h',
                     labels={'person_id': 'Contagem de Jogadores', 'school': 'Escola'},
                     title='Top 20 Escolas por Contagem de Jogadores',
                     height=400, template='plotly_dark')
        fig.update_layout(margin=dict(l=0, r=0, t=40, b=0), plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
        return pn.pane.Plotly(fig, config={'displayModeBar': True}, height=400)

    @pn.depends(school_selector.param.value, position_selector.param.value)
    def grafico_position(schools, positions):
        if not schools or not positions: 
            return go.Figure().update_layout(template='plotly_dark', title='Contagem de Jogadores por Posição', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
            
        df = filtrar_jogadores(schools, positions)
        contagem_position = df.groupby('position').agg({'person_id': 'count'}).reset_index()
        contagem_position = contagem_position.sort_values('person_id', ascending=False)
        
        fig = px.bar(contagem_position, x='position', y='person_id',
                     labels={'person_id': 'Contagem de Jogadores', 'position': 'Posição'},
                     title='Contagem de Jogadores por Posição',
                     height=300, template='plotly_dark')
        fig.update_layout(margin=dict(l=0, r=0, t=40, b=0), plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
        return pn.pane.Plotly(fig, config={'displayModeBar': True}, height=300)

    # --- Layout da Aba ---
    return pn.Column(
        pn.pane.Markdown('## Análise de Jogadores'),
        pn.Row(
            pn.Card(
                school_selector,
                position_selector,
                title="Filtros",
                width=300,
                # 'background' FOI REMOVIDO (CORRIGIDO)
            ),
            pn.Column(
                grafico_school,
                grafico_position,
                sizing_mode='stretch_width'
            ),
        ),
        css_classes=['glass-pane'],
        sizing_mode='stretch_width'
    )

def create_game_analysis_tab(data):
    game = data['game_with_names']
    team_df = data['team_df_processed']
    
    data_min = game['game_date'].min()
    data_max = game['game_date'].max()
    start_default = max(pd.Timestamp('2020-01-01'), data_min)
    end_default = data_max

    # --- Widgets ---
    date_range_slider = pn.widgets.DateRangeSlider(
        name='Intervalo de Data dos Jogos',
        start=data_min, end=data_max,
        value=(start_default, end_default),
        format='%d/%m/%Y',
        sizing_mode='stretch_width'
    )

    def jogos_filtrados(start, end):
        start = pd.Timestamp(start)
        end = pd.Timestamp(end)
        mask = (game['game_date'] >= start) & (game['game_date'] <= end)
        return game.loc[mask, 'nome_jogo'].tolist()

    initial_options = jogos_filtrados(start_default, end_default)
    jogo_selector = pn.widgets.Select(
        name='Filtrar por Jogo',
        options=initial_options,
        value=initial_options[0] if initial_options else None,
        sizing_mode='stretch_width'
    )

    # --- Lógica de Atualização de Filtro Vinculado ---
    def atualizar_jogos(event):
        opcoes = jogos_filtrados(event.new[0], event.new[1])
        jogo_selector.options = opcoes
        if opcoes:
            jogo_selector.value = opcoes[0]
        else:
            jogo_selector.value = None

    date_range_slider.param.watch(atualizar_jogos, 'value')

    # --- Função Reativa do Painel ---
    @pn.depends(jogo_selector.param.value)
    def painel_jogo(nome_jogo):
        if not nome_jogo:
            return pn.pane.Alert("Nenhum jogo selecionado para o intervalo de datas.", alert_type='warning')
            
        row = game[game['nome_jogo'] == nome_jogo].iloc[0]
        
        home_team_row = team_df[team_df['team_id'] == row['team_id_home']]
        away_team_row = team_df[team_df['team_id'] == row['team_id_away']]
        
        home_team = home_team_row.iloc[0]['nickname'] if not home_team_row.empty else 'N/A'
        away_team = away_team_row.iloc[0]['nickname'] if not away_team_row.empty else 'N/A'

        pts_home = row['pts_home']
        pts_away = row['pts_away']
        total_pontos = pts_home + pts_away

        style_card = """
            background:rgba(0,30,60,0.7); border-radius:32px; padding:32px 16px; color:#fff;
            box-shadow:0 4px 24px #0006; text-align:center; font-family:sans-serif;
            height:150px; display:flex; flex-direction:column; justify-content:center;
            align-items:center; width:250px;
        """
        style_title = 'font-size:2.5em;font-weight:bold;text-shadow:0 0 8px #00f;'
        style_label = 'font-size:1.2em;margin-bottom:8px;'
        style_points = 'font-size:2.5em;font-weight:bold;'
        style_total = 'font-size:2em;font-weight:bold;'

        return pn.Column(
            pn.Row(
                pn.Column(
                    pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Time da Casa</div><div style="{style_title}">{home_team}</div></div>'),
                    pn.Spacer(height=20),
                    pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Pontos</div><div style="{style_points}">{pts_home:,.0f}</div></div>'),
                ),
                pn.Column(
                    pn.Spacer(height=40),
                    pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Total de Pontos</div><div style="{style_total}">{total_pontos:,.0f}</div></div>'),
                ),
                pn.Column(
                    pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Time Visitante</div><div style="{style_title}">{away_team}</div></div>'),
                    pn.Spacer(height=20),
                    pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Pontos</div><div style="{style_points}">{pts_away:,.0f}</div></div>'),
                ),
                css_classes=['jogo-row'],
                sizing_mode='stretch_width',
                align='center'
            )
        )

    # --- Layout da Aba ---
    return pn.Column(
        pn.pane.Markdown('## Análise de Jogo Individual'),
        pn.Card(
            date_range_slider,
            jogo_selector,
            title="Filtros",
            collapsed=False
            # 'background' FOI REMOVIDO (CORRIGIDO)
        ),
        pn.Spacer(height=20),
        painel_jogo,
        css_classes=['glass-pane'],
        sizing_mode='stretch_width',
        align='center'
    )

def create_physical_profile_tab(data):
    df_info = data['player_info']
    df_player = data['player']
    
    times = sorted(df_info['team_name'].dropna().unique().tolist())
    posicoes = sorted(df_info['position'].dropna().unique().tolist())

    # --- Widgets ---
    time_selector = pn.widgets.Select(name='Time', options=['Todos'] + times, value='Todos', width=300)
    pos_selector = pn.widgets.Select(name='Posição', options=['Todos'] + posicoes, value='Todos', width=300)

    # --- Funções Reativas ---
    @pn.depends(time_selector.param.value, pos_selector.param.value)
    def filtrar(time, pos):
        df = df_info.copy()
        if time != 'Todos':
            df = df[df['team_name'] == time]
        if pos != 'Todos':
            df = df[df['position'] == pos]
        return df

    @pn.depends(time_selector.param.value, pos_selector.param.value)
    def grafico_altura(time, pos):
        df = filtrar(time, pos)
        fig = px.histogram(df, x='altura_m', nbins=20, labels={'altura_m': 'Altura (m)'}, title='Distribuição de Jogadores por Altura', template='plotly_dark')
        fig.update_layout(height=350, plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
        fig.update_yaxes(title='Contagem de Jogadores')
        return fig

    @pn.depends(time_selector.param.value, pos_selector.param.value)
    def grafico_peso(time, pos):
        df = filtrar(time, pos)
        fig = px.histogram(df, x='peso_kg', nbins=20, labels={'peso_kg': 'Peso (libras)'}, title='Distribuição de Jogadores por Peso', template='plotly_dark')
        fig.update_layout(height=350, plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
        fig.update_yaxes(title='Contagem de Jogadores')
        return fig

    @pn.depends(time_selector.param.value, pos_selector.param.value)
    def grafico_dispersao(time, pos):
        df = filtrar(time, pos)
        df = df.merge(df_player[['id', 'full_name']], left_on='person_id', right_on='id', how='left')
        fig = px.scatter(
            df, x='altura_m', y='peso_kg', color='position',
            hover_data={'full_name': True, 'position': True, 'altura_m': True, 'peso_kg': True},
            labels={'altura_m': 'Altura (m)', 'peso_kg': 'Peso (libras)', 'position': 'Posição'},
            title='Altura vs. Peso por Posição',
            height=400, template='plotly_dark'
        )
        fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
        return fig

    # --- Layout da Aba ---
    return pn.Column(
        pn.pane.Markdown('## Perfil Físico dos Jogadores'),
        pn.Row(
            pn.Column(
                pn.pane.Markdown('### Distribuição por Altura'),
                grafico_altura,
                sizing_mode='stretch_width'
            ),
            pn.Column(
                pn.pane.Markdown('### Distribuição por Peso'),
                grafico_peso,
                sizing_mode='stretch_width'
            ),
            pn.Card(
                pn.pane.Markdown('<b>Time</b>'), time_selector,
                pn.pane.Markdown('<b>Posição</b>'), pos_selector,
                title="Filtros",
                width=350
                # 'background' FOI REMOVIDO (CORRIGIDO)
            ),
        ),
        pn.Row(
            pn.Column(
                pn.pane.Markdown('### Altura vs. Peso por Posição'),
                grafico_dispersao,
                sizing_mode='stretch_width'
            ),
        ),
        css_classes=['glass-pane'],
        sizing_mode='stretch_width'
    )

# --- 5. Montagem Final do Dashboard -------------------------------------

def create_dashboard():
    all_data = load_data()
    if all_data is None:
        return pn.pane.Alert("Falha ao carregar dados. Verifique os arquivos CSV.", alert_type='danger')

    # Dicionário de páginas
    pages = {
        "Início": create_home_tab(),
        "Visão Geral": create_overview_tab(all_data),
        "Análise de Times": create_team_analysis_tab(all_data),
        "Análise Geográfica": create_geo_tab(all_data),
        "Análise de Jogadores": create_player_analysis_tab(all_data),
        "Análise de Jogos": create_game_analysis_tab(all_data),
        "Perfil Físico": create_physical_profile_tab(all_data),
    }

    # Área principal dinâmica
    main_area = pn.Column(pages["Início"], sizing_mode='stretch_width')

    def show_page(event):
        main_area.clear()
        main_area.append(pages[event.obj.name])

    menu_items = []
    for name in pages:
        btn = pn.widgets.Button(name=name, button_type='primary', width=220)
        btn.on_click(show_page)
        menu_items.append(btn)

    menu = pn.Column(*menu_items, sizing_mode="fixed", width=240)

    # Template principal
    dashboard = pn.template.FastListTemplate(
        site="Dashboard NBA",
        title="",
        sidebar=[
            pn.pane.Markdown("## Menu de Navegação"),
            menu
        ],
        main=[main_area],
        theme='dark',
        accent_base_color="#007BFF"
    )
    
    return dashboard

# --- 6. Servir o Dashboard ----------------------------------------------
create_dashboard().servable()